# Study of obesity prediction dataset

This notebook was created on google collab

## Import packages

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay

ModuleNotFoundError: No module named 'sklearn'

# Import data
Data are taken from Kaggle.com

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adeniranstephen/obesity-prediction-dataset")

print("Path to dataset files:", path)

c:\Users\flavi\Documents\Projets_github\La_Plage\la_plage\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 32.7k/32.7k [00:00<00:00, 623kB/s]

Extracting files...
Path to dataset files: C:\Users\flavi\.cache\kagglehub\datasets\adeniranstephen\obesity-prediction-dataset\versions\1


In [4]:
df = pd.read_csv(path + '/ObesityDataSet_raw_and_data_sinthetic.csv')

In [5]:
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   int64  
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

* Gender – Male or Female.
* Age – The person’s age in years.
* Height – Height in meters.
* Weight – Weight in kilograms.
* family_history_with_overweight – Whether the person has a family history of * * being overweight (yes/no).
* FAVC – If the person frequently consumes high-calorie foods (yes/no).
* FCVC – Frequency of vegetable consumption (scale from 1 to 3).
* NCP – Number of main meals per day.
* CAEC – Frequency of consuming food between meals (Never, Sometimes, Frequently, Always).
* SMOKE – Whether the person smokes (yes/no).
* CH2O – Daily water intake (scale from 1 to 3).
* SCC – If the person monitors their calorie intake (yes/no).
* FAF – Physical activity frequency (scale from 0 to 3).
* TUE – Time spent using technology (scale from 0 to 3).
* CALC – Frequency of alcohol consumption (Never, Sometimes, Frequently, Always).
* MTRANS – Main mode of transportation (Automobile, Bike, Motorbike, Public Transportation, Walking).
* **NObeyesdad – Obesity level (Insufficient Weight, Normal Weight, Overweight Level I, Overweight Level II, Obesity Type I, Obesity Type II, Obesity Type III)**

In a prediction model, a target in this dataset could be NObeyesdad.
We have 2111 idividuals.


## Explore data

Remove duplicated an get an overview

In [7]:
# See if there are duplicated
df[df.duplicated()]
# yes

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
98,Female,21,1.52,42.0,no,no,3.0,1.0,Frequently,no,1.0,no,0.0,0.0,Sometimes,Public_Transportation,Insufficient_Weight
106,Female,25,1.57,55.0,no,yes,2.0,1.0,Sometimes,no,2.0,no,2.0,0.0,Sometimes,Public_Transportation,Normal_Weight
174,Male,21,1.62,70.0,no,yes,2.0,1.0,no,no,3.0,no,1.0,0.0,Sometimes,Public_Transportation,Overweight_Level_I
179,Male,21,1.62,70.0,no,yes,2.0,1.0,no,no,3.0,no,1.0,0.0,Sometimes,Public_Transportation,Overweight_Level_I
184,Male,21,1.62,70.0,no,yes,2.0,1.0,no,no,3.0,no,1.0,0.0,Sometimes,Public_Transportation,Overweight_Level_I
209,Female,22,1.69,65.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Sometimes,Public_Transportation,Normal_Weight
309,Female,16,1.66,58.0,no,no,2.0,1.0,Sometimes,no,1.0,no,0.0,1.0,no,Walking,Normal_Weight
460,Female,18,1.62,55.0,yes,yes,2.0,3.0,Frequently,no,1.0,no,1.0,1.0,no,Public_Transportation,Normal_Weight
467,Male,22,1.74,75.0,yes,yes,3.0,3.0,Frequently,no,1.0,no,1.0,0.0,no,Automobile,Normal_Weight
496,Male,18,1.72,53.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,2.0,Sometimes,Public_Transportation,Insufficient_Weight


In [8]:
# duplicata deleted
df = df[~df.duplicated()]

In [9]:
# check null values # if null values : imputation (or deletion)
df.isnull().sum()

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

Add of variable BMI (numerical).
Check of consistency between BMI and obesity level (graphically).

In [ ]:
# french IMC
df['BMI'] = df['Weight']/(df['Height']**2)


In [ ]:
sns.scatterplot(df,x=range(2087), y ='BMI', hue= 'NObeyesdad')

# Add line on classical thresholds
sns.lineplot(x=range(2100), y=18.5)
sns.lineplot(x=range(2100), y=25)
sns.lineplot(x=range(2100), y=30)
sns.lineplot(x=range(2100), y=35)
sns.lineplot(x=range(2100), y=40)


It is not a perfect match but a good aprroximation.

Let's continue our exploration.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2087 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2087 non-null   object 
 1   Age                             2087 non-null   int64  
 2   Height                          2087 non-null   float64
 3   Weight                          2087 non-null   float64
 4   family_history_with_overweight  2087 non-null   object 
 5   FAVC                            2087 non-null   object 
 6   FCVC                            2087 non-null   float64
 7   NCP                             2087 non-null   float64
 8   CAEC                            2087 non-null   object 
 9   SMOKE                           2087 non-null   object 
 10  CH2O                            2087 non-null   float64
 11  SCC                             2087 non-null   object 
 12  FAF                             2087 no

In [11]:
df.describe() # for numerical values

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2087.000000,2087.000000,2087.000000,2087.000000,2087.000000,2087.000000,2087.000000,2087.000000
mean,24.356493,1.702616,86.858706,2.421409,2.701203,2.004792,1.012827,0.663031
std,6.379977,0.093251,26.190838,0.534807,0.764655,0.608281,0.853496,0.608152
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,20.000000,1.630000,66.000000,2.000000,2.700000,1.590000,0.125000,0.000000
50%,23.000000,1.700000,83.100000,2.400000,3.000000,2.000000,1.000000,0.631000
75%,26.000000,1.770000,108.015000,3.000000,3.000000,2.465000,1.680000,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


In [12]:
df.select_dtypes(include='object').columns # categorical values


Index(['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE',
       'SCC', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [13]:
df.select_dtypes(exclude='object').columns

Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE'], dtype='object')

In [16]:
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
  df[col] = df[col].astype('category')
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2087 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Gender                          2087 non-null   category
 1   Age                             2087 non-null   int64   
 2   Height                          2087 non-null   float64 
 3   Weight                          2087 non-null   float64 
 4   family_history_with_overweight  2087 non-null   category
 5   FAVC                            2087 non-null   category
 6   FCVC                            2087 non-null   float64 
 7   NCP                             2087 non-null   float64 
 8   CAEC                            2087 non-null   category
 9   SMOKE                           2087 non-null   category
 10  CH2O                            2087 non-null   float64 
 11  SCC                             2087 non-null   category
 12  FAF                      

## One-dimensional analysis

NObeyesdad (obesity level) studing.

In [ ]:
# Number of individuals per obesity level
df['NObeyesdad'].value_counts()

In [ ]:
# Repartition of individuals per obesity level
fig, ax = plt.subplots()
ax.pie(df['NObeyesdad'].value_counts(), labels=df['NObeyesdad'].value_counts().index, autopct='%1.1f%%')
plt.show()

In [ ]:
# Gender repartition

gender = df['Gender'].value_counts()
plt.bar(gender.index, gender.values, color=['blue', 'pink'])
plt.title('Man and woman repartition')
plt.xlabel('Gender')
plt.ylabel('Numeber')

plt.show()

In [ ]:
# age repartition
# mean = 24.315964
# Q_0,5 = 23
plt.subplot(1,2, 1)

plt.hist(df['Age'], bins=10, density = True)
plt.title('Age distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')

plt.subplot(1,2, 2)
plt.boxplot(df['Age'],patch_artist=True)
plt.title('Age distribution')
plt.xlabel('Age')

plt.show()

## Two dimensions analysis

In [ ]:
# family impact over weiht condition
pd.crosstab(df['NObeyesdad'], df['family_history_with_overweight'])

In [ ]:
# influence of weiht on height still present at each obestity level
# we could expect this, seems logical
sns.scatterplot(df, x='Height', y = 'Weight', hue= 'NObeyesdad') # seaborn is better to colorized point on scatter. with matplolib we have to loop.

In [ ]:
# influence of an trasportation mode on level on BMI
plt.figure(figsize=(10, 5))
sns.boxplot(x='MTRANS', y='BMI', data=df, palette='viridis')

In [ ]:
# influence of calories monitoring on BMI
plt.figure(figsize=(10, 5))
sns.boxplot(x='SCC', y='BMI', data=df, palette='viridis')

In [ ]:
# influence of smoking on BMI
plt.figure(figsize=(10, 5))
sns.boxplot(x='SMOKE', y='BMI', data=df, palette='viridis')

Boxplot are almost aligned, so we can consider smoking do not influ a lot on BMI we will not consider it later.

Correlation matrix of quantitative variables.

In [ ]:
plt.matshow(df.select_dtypes(exclude='object').corr(), cmap='seismic',vmin=-1, vmax=1)
plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=6, rotation=65)
plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=6)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);
plt.show()

In [ ]:
df.select_dtypes(exclude='object')

## Principal component analysis


In [ ]:
# Selection of numerical features
X = df[["FCVC","NCP","CH2O","FAF", "TUE", "Height"]]
X.head()

In [ ]:
print(X.nunique()) # check not categorical or ordinal even if weird

In [ ]:
X['NObeyesdad'] = df['NObeyesdad']
sns.pairplot(X, hue= 'NObeyesdad')


Work with centred and reduce values to consider the average individual and removed problems dur to differnts units.

In [ ]:
# use standard scaler provided by sklearn
X = X.drop(['NObeyesdad'], axis=1)
X_sc = StandardScaler().fit_transform(X)

In [ ]:
pca = PCA()
X_pca = pca.fit_transform(X)
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_.cumsum(), marker='o')
plt.xlabel('Number of principal component')
plt.ylabel('Cumsum explain variance')
plt.title('Scree Plot')
plt.show()

We selected 4 principals components. to get more than 80%

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.singular_values_

In [ ]:
# Get the explained variance ratios for the selected components
explained_variances = pca.explained_variance_ratio_

# Get the principal component vectors (also known as loadings)
loadings = pca.components_.T * np.sqrt(explained_variances)

# Set figsize and other layout parameters
plt.figure(figsize=(8, 8))
plt.title('Correlation Circle Plot')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
# Add variable labels to the plot
for i, feature in enumerate(X.columns):

    # Add the name of the variable near the arrow
    plt.annotate(feature, # variable name
                 (loadings[i, 0],
                  loadings[i, 1]),
                 color='red')


    # Add an arrow representing the variable on the new axis
    plt.arrow(0, 0,
              loadings[i, 0],
              loadings[i, 1],
              color='black',
              alpha=0.7,
              width=0.01,
             )

# Fix x-axis between -1 and 1 is important for better visualization
plt.xlim(-1,1)
plt.ylim(-1,1)

# Add grid in the background
plt.grid(True)

# Display the chart
plt.show()

In [ ]:

# Set figsize and other layout parameters
plt.figure(figsize=(8, 8))
plt.title('Correlation Circle Plot')
plt.xlabel('Principal Component 3')
plt.ylabel('Principal Component 4')
# Add variable labels to the plot
for i, feature in enumerate(X.columns):

    # Add the name of the variable near the arrow
    plt.annotate(feature, # variable name
                 (loadings[i, 2],
                  loadings[i, 3]),
                 color='red')


    # Add an arrow representing the variable on the new axis
    plt.arrow(0, 0,
              loadings[i, 2],
              loadings[i, 3],
              color='black',
              alpha=0.7,
              width=0.01,
             )

# Fix x-axis between -1 and 1 is important for better visualization
plt.xlim(-1,1)
plt.ylim(-1,1)

# Add grid in the background
plt.grid(True)

# Display the chart
plt.show()

In [ ]:
X_pca_df = pd.DataFrame(X_pca, columns=['Dim1', 'Dim2', 'Dim3', 'Dim4','Dim5', 'Dim6'])
X_pca_df['NObeyesdad'] = df['NObeyesdad']

In [ ]:
sns.scatterplot(X_pca_df, x='Dim1', y= 'Dim2', hue= 'NObeyesdad')
#We cannot distinguidh classes
# it was expected because 2 dim is not enought.

In [ ]:
''' scatter = ax.scatter(
    X_reduced[:, 0],
    X_reduced[:, 1],
    X_reduced[:, 2],
    c=iris.target,
    s=40,
)

ax.set(
    title="First three PCA dimensions",
    xlabel="1st Eigenvector",
    ylabel="2nd Eigenvector",
    zlabel="3rd Eigenvector",
)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.zaxis.set_ticklabels([])

# Add a legend
legend1 = ax.legend(
    scatter.legend_elements()[0],
    iris.target_names.tolist(),
    loc="upper right",
    title="Classes",
)
ax.add_artist(legend1)

plt.show() '''

## t-sne approach

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# Appliquer t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X)

# Créer un DataFrame pour faciliter la manipulation
X_tsne_df = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2'])
X_tsne_df['NObeyesdad'] = df['NObeyesdad']

# Tracer les résultats
plt.figure(figsize=(8, 6))
sns.scatterplot(X_tsne_df, x='TSNE1', y='TSNE2', hue='NObeyesdad')

## Prediction


In [ ]:
encoding_freq = {"no":0, "Sometimes":1, "Frequently":2, "Always":3}
df['CAEC'] = df['CAEC'].map(encoding_freq) # food between meal
df['CALC'] = df['CALC'].map(encoding_freq) # alcool consuption

In [ ]:
encoding_yes_no = {"no":0, "yes":1}
df['family_history_with_overweight'] = df['family_history_with_overweight'].map(encoding_yes_no)
df['FAVC'] = df['FAVC'].map(encoding_yes_no)
# df['SMOKE'] = df['SMOKE'].map(Encoding_yes_no) not selected

In [ ]:
encoding_transport = {"Automobile":4, "Motorbike":2, "Bike":1, "Public_Transportation":3, "Walking":0}
df['MTRANS'] = df['MTRANS'].map(encoding_transport)

In [ ]:
X = df[['CAEC', 'CALC', 'family_history_with_overweight', 'FAVC', 'MTRANS',"FCVC","NCP","CH2O","FAF", "TUE", "Height"]]
X.head()

In [ ]:
### Logistique regression

In [ ]:
df['Obesity'] = df['NObeyesdad'].apply(lambda x: '1' if x in ['Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III'] else '0')
df['Obesity']

In [ ]:
df['Obesity'].value_counts()

In [ ]:
Y = df['Obesity']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
X_train

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
model = LogisticRegression().fit(X_train_scaled, y_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)
predictions = model.predict(X_test_scaled)

In [ ]:
accuracy_score(y_test, predictions)

In [ ]:
ConfusionMatrixDisplay.from_estimator(model, X_test_scaled, y_test)